In [6]:
import numpy as np
import pandas as pd

In [7]:
with open('data/ass3.pickle', 'rb') as handle:
    data = pd.read_pickle(handle)

X_train, y_train = data['train']
X_dev, y_dev = data['dev']

In [8]:

print('Train data:')
rows_train, columns_train = X_train.shape
print(f'Number of Rows: {rows_train}, Number of Coloumns: {columns_train}')

print('Dev data:')
rows_dev, columns_dev = X_dev.shape
print(f'Number of Rows: {rows_dev}, Number of Coloumns: {columns_dev}')


Train data:
Number of Rows: 48226, Number of Coloumns: 13
Dev data:
Number of Rows: 24114, Number of Coloumns: 13


In [5]:
from lazypredict.Supervised import LazyRegressor

reg = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = reg.fit(X_train, X_dev, y_train, y_dev)

print(models)

C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
100%|██████████| 42/42 [12:16<00:00, 17.54s/it] 

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
ExtraTreesRegressor                          0.97       0.97  1.72       34.02
RandomForestRegressor                        0.96       0.96  1.91       41.66
BaggingRegressor                             0.95       0.95  2.04        4.80
XGBRegressor                                 0.95       0.95  2.14        6.40
HistGradientBoostingRegressor                0.93       0.93  2.50        4.32
LGBMRegressor                                0.93       0.93  2.50        1.05
MLPRegressor                                 0.92       0.92  2.78      124.31
DecisionTreeRegressor                        0.92       0.92  2.78        1.22
ExtraTreeRegressor                           0.90       0.90  2.98        0.64
GradientBoostingRegressor                    0.90       0.90  3.00       13.46
KNeighborsRegressor                          0.89   

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

def select_k_best(k, X, y):
    # k_best = SelectKBest(score_func=f_regression, k=k)
    k_best = SelectKBest(mutual_info_regression, k=k)
    k_best.fit(X, y)
    return (k_best.transform(X), k_best.transform(X_dev))

In [10]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

def cross_validation(reg, X, y, k):
    sc = StandardScaler()
    return round(cross_val_score(reg, sc.fit_transform(X), y, cv=k).mean(), 3)

def fit_predict(regressor, X_train, y_train, X_dev):
    sc = StandardScaler()
    scaled_X_train = sc.fit_transform(X_train)
    regressor.fit(scaled_X_train, y_train)

    train_score = mean_absolute_error(regressor.predict(scaled_X_train), y_train)
    dev_score = mean_absolute_error(regressor.predict(sc.transform(X_dev)), y_dev)
    over_fitting = train_score - dev_score

    return f'train score: {round(train_score, 3)} test score {round(dev_score, 3)} overfit {round(over_fitting, 3)}'

def fit_predict_poly(regressor, X_train, y_train, X_dev, y_dev):
    transformed_train = PolynomialFeatures(2).fit_transform(X_train)
    transformed_dev = PolynomialFeatures(2).fit_transform(X_dev)
    sc = StandardScaler()
    scaled_X_train = sc.fit_transform(transformed_train)

    regressor.fit(scaled_X_train, y_train)

    train_score = mean_absolute_error(regressor.predict(scaled_X_train), y_train)
    dev_score = mean_absolute_error(regressor.predict(sc.transform(transformed_dev)), y_dev)
    over_fitting = train_score - dev_score

    return f'train score: {round(train_score, 3)} test score {round(dev_score, 3)} overfit {round(over_fitting, 3)}'

In [12]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

def check_models(X_train, y, k):
    for reg in regs:
        print(reg)
        # for i in range(1,14):
        #     X_train, X_dev = select_k_best(i, X, y)
        print(f'cv {cross_validation(regs[reg], np.array(X_train), np.array(y), k)} Predict {fit_predict(regs[reg], X_train, y, X_dev)} Polynomial {fit_predict_poly(regs[reg], X_train, y, X_dev, y_dev)}')

regs = {
    # 'Random Forest':RandomForestRegressor(random_state = 42, bootstrap=True, max_depth=70, min_samples_leaf=4, min_samples_split=10, n_estimators=400)}
    # 'AdaBoost':AdaBoostRegressor(random_state=42),
    # 'Histogram Gradient Boosting':HistGradientBoostingRegressor(random_state = 42),
    # 'XGBoost':XGBRegressor(random_state = 42),
    # 'Light GBM':LGBMRegressor(random_state = 42),
    'ExtraTreesRegressor' : ExtraTreesRegressor(random_state=42)
    # 'Bagging Regressor':BaggingRegressor(random_state=42)
    # 'SVR':SVR()
    }

# print('Regular Data:')
check_models(X_train, y_train, 5)
# print('Undersampled Data:')
# check_models(x_rus, y_rus, 5)
# print('Oversampled Data:')
# check_models(x_ros, y_ros, 5)

ExtraTreesRegressor
cv 0.964 Predict train score: 0.0 test score 1.24 overfit -1.24 Polynomial train score: 0.0 test score 1.266 overfit -1.266


In [21]:
from sklearn.model_selection import RandomizedSearchCV

#The function to measure the quality of a split
criterion = ['squared_error', 'absolute_error']
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
rf_grid =  {'criterion' : criterion,
            'n_estimators': n_estimators,
            'max_features': max_features,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = RandomForestRegressor(random_state=42), param_distributions = rf_grid)
rf_random.fit(X_train, y_train)
print(rf_random.best_params_)
print(rf_random.best_estimator_)

KeyboardInterrupt: 

In [17]:
xgb_grid = {
    "learning_rate"     : [0.05, 0.1, 0.15, 0.2, 0.25, 0.3] ,
    "max_depth"         : [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight"  : [1, 3, 5, 7],
    "gamma"             : [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree"  : [0.3, 0.4, 0.5, 0.7]
}

xgb_random = RandomizedSearchCV(XGBRegressor(random_state=42), param_distributions=xgb_grid, n_iter=50, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)
xgb_random.fit(X_train, y_train)
print(xgb_random.best_params_)
print(xgb_random.best_estimator_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


{'min_child_weight': 3, 'max_depth': 4, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.5}
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.3, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=3,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)


In [18]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

lgbm_grid = {'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

lgbm_random = RandomizedSearchCV(LGBMRegressor(random_state=42), param_distributions=lgbm_grid, n_iter=50, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)

lgbm_random.fit(X_train, y_train)
print(lgbm_random.best_params_)
print(lgbm_random.best_estimator_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'colsample_bytree': 0.7335544514009392, 'min_child_samples': 169, 'min_child_weight': 1, 'num_leaves': 24, 'reg_alpha': 5, 'reg_lambda': 5, 'subsample': 0.3224398548522627}
LGBMRegressor(colsample_bytree=0.7335544514009392, min_child_samples=169,
              min_child_weight=1, num_leaves=24, random_state=42, reg_alpha=5,
              reg_lambda=5, subsample=0.3224398548522627)


In [19]:
hgb_grid = {
    'learning_rate': (0.01, 0.1, 1, 10),
    'max_leaf_nodes': (3, 10, 30, 60),
    'min_samples_leaf': (5, 10, 20, 40),
    'max_depth': (5, 6, 7, 8)}

hgb_random = RandomizedSearchCV(HistGradientBoostingRegressor(random_state=42), param_distributions=hgb_grid, n_iter=50, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)

hgb_random.fit(X_train, y_train)
print(hgb_random.best_params_)
print(hgb_random.best_estimator_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'min_samples_leaf': 40, 'max_leaf_nodes': 3, 'max_depth': 8, 'learning_rate': 10}
HistGradientBoostingRegressor(learning_rate=10, max_depth=8, max_leaf_nodes=3,
                              min_samples_leaf=40, random_state=42)


c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
